In [82]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

import matlab.engine as engi
import matlab as mat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from pyearth import Earth

from src import SMOTE
from src import CFS
from src import metrices_V2 as metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt

In [83]:
def load_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    commit_guru_df = pd.read_csv(commit_guru_path)
#     print(commit_guru_df.columns)
    commit_guru_df = commit_guru_df.drop(labels = ['parent_hashes','author_name','author_name',
                                                   'author_email','author_date','fileschanged',
                                                   'author_date_unix_timestamp', 'commit_message','commit_hash',
                                                  'classification', 'fix','fixes',],axis=1)

    df = commit_guru_df
    df.rename(columns={"contains_bug": "Bugs"},inplace=True)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    y.fillna(False,inplace=True)
    df['Bugs'] = y
    df.dropna(inplace=True)
    cols = df.columns.tolist()
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    return X,y

def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

In [79]:
def run_self(project):
    X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    loc = X_test['la'] + X_test['lt']
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = RandomForestClassifier()
    clf.fit(X_train,y_train)
    predicted = clf.predict(X_test)
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa

In [89]:
def run_self_CFS(project):
    X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    loc = X_test['la'] + X_test['lt']
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    df_smote,cols = apply_cfs(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = RandomForestClassifier()
    clf.fit(X_train,y_train)
    predicted = clf.predict(X_test[cols[:-1]])
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa

In [90]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

In [91]:
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
pci_20_list = {}
ifa_list = {}
for project in projects:
    try:
        if project == '.DS_Store':
            continue
    #     if project != 'guice':
    #         continue
        print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
        recall,precision,pf,f1,g_score,auc,pci_20,ifa = run_self_CFS(project)
        recall_list[project] = recall
        precision_list[project] = precision
        pf_list[project] = pf
        f1_list[project] = f1
        g_list[project] = g_score
        auc_list[project] = auc
        pci_20_list[project] = pci_20
        ifa_list[project] = ifa
    except Exception as e:
        print(e)
        continue
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list
final_result['auc'] = auc_list
final_result['pci_20'] = pci_20_list
final_result['ifa'] = ifa_list
with open('results/Performance/process_1000_CFS.pkl', 'wb') as handle:
    pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)

+++++++++++++++++   org.alloytools.alloy  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.97      1.00      0.99        38
        True       0.00      0.00      0.00         1

    accuracy                           0.97        39
   macro avg       0.49      0.50      0.49        39
weighted avg       0.95      0.97      0.96        39

+++++++++++++++++   qpython  +++++++++++++++++
index 1 is out of bounds for axis 0 with size 1
+++++++++++++++++   friendlychat-android  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.92      1.00      0.96        11
        True       0.00      0.00      0.00         1

    accuracy                           0.92        12
   macro avg       0.46      0.50      0.48        12
weighted avg       0.84      0.92      0.88        12

+++++++++++++++++   paho.mqtt.android  +++++++++++++++++
              precision    recall  f1-score   support

       False       

index 1 is out of bounds for axis 0 with size 1
+++++++++++++++++   Dachshund-Tab-Layout  +++++++++++++++++
index 1 is out of bounds for axis 0 with size 1
+++++++++++++++++   Pix-EzViewer  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.97      1.00      0.98        64
        True       0.00      0.00      0.00         2

    accuracy                           0.97        66
   macro avg       0.48      0.50      0.49        66
weighted avg       0.94      0.97      0.95        66

+++++++++++++++++   FastScroll  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.88      1.00      0.94        15
        True       0.00      0.00      0.00         2

    accuracy                           0.88        17
   macro avg       0.44      0.50      0.47        17
weighted avg       0.78      0.88      0.83        17

+++++++++++++++++   recyclerviewhelper  +++++++++++++++++
index 1 is out of bounds for a

              precision    recall  f1-score   support

       False       0.97      0.89      0.93        35
        True       0.00      0.00      0.00         1

    accuracy                           0.86        36
   macro avg       0.48      0.44      0.46        36
weighted avg       0.94      0.86      0.90        36

+++++++++++++++++   Intro-To-RxJava  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.98      1.00      0.99       113
        True       0.00      0.00      0.00         2

    accuracy                           0.98       115
   macro avg       0.49      0.50      0.50       115
weighted avg       0.97      0.98      0.97       115

+++++++++++++++++   Swipeable-Cards  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.96      1.00      0.98        22
        True       1.00      0.50      0.67         2

    accuracy                           0.96        24
   macro avg      

              precision    recall  f1-score   support

       False       0.99      0.92      0.95        73
        True       0.00      0.00      0.00         1

    accuracy                           0.91        74
   macro avg       0.49      0.46      0.48        74
weighted avg       0.97      0.91      0.94        74

+++++++++++++++++   io2015-codelabs  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.94      1.00      0.97        31
        True       1.00      0.33      0.50         3

    accuracy                           0.94        34
   macro avg       0.97      0.67      0.73        34
weighted avg       0.94      0.94      0.93        34

+++++++++++++++++   FlyoutMenus  +++++++++++++++++
index 1 is out of bounds for axis 0 with size 1
+++++++++++++++++   react-native-share-extension  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.96      0.82      0.88        28
        True   

              precision    recall  f1-score   support

       False       0.98      0.99      0.98       134
        True       0.00      0.00      0.00         3

    accuracy                           0.96       137
   macro avg       0.49      0.49      0.49       137
weighted avg       0.96      0.96      0.96       137

+++++++++++++++++   spring-mvc-showcase  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.81      0.92      0.86        24
        True       0.00      0.00      0.00         5

    accuracy                           0.76        29
   macro avg       0.41      0.46      0.43        29
weighted avg       0.67      0.76      0.71        29

+++++++++++++++++   RMSwitch  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.81      1.00      0.90        13
        True       1.00      0.25      0.40         4

    accuracy                           0.82        17
   macro avg       0.

...

              precision    recall  f1-score   support

       False       0.78      0.80      0.79       182
        True       0.63      0.60      0.62       103

    accuracy                           0.73       285
   macro avg       0.71      0.70      0.70       285
weighted avg       0.73      0.73      0.73       285

+++++++++++++++++   RSyntaxTextArea  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.82      0.85      0.84       171
        True       0.74      0.70      0.72       105

    accuracy                           0.79       276
   macro avg       0.78      0.77      0.78       276
weighted avg       0.79      0.79      0.79       276

+++++++++++++++++   facebook-android-sdk  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.91      0.90      0.91       394
        True       0.62      0.65      0.63        99

    accuracy                           0.85       493
   macro avg 

              precision    recall  f1-score   support

       False       0.87      0.90      0.89       387
        True       0.59      0.51      0.55       106

    accuracy                           0.82       493
   macro avg       0.73      0.71      0.72       493
weighted avg       0.81      0.82      0.81       493

+++++++++++++++++   boon  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.82      0.80      0.81       253
        True       0.56      0.59      0.58       110

    accuracy                           0.74       363
   macro avg       0.69      0.69      0.69       363
weighted avg       0.74      0.74      0.74       363

+++++++++++++++++   spring-data-solr  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.90      0.88      0.89       212
        True       0.80      0.83      0.81       124

    accuracy                           0.86       336
   macro avg       0.85     

              precision    recall  f1-score   support

       False       0.84      0.83      0.83       263
        True       0.64      0.65      0.64       121

    accuracy                           0.77       384
   macro avg       0.74      0.74      0.74       384
weighted avg       0.78      0.77      0.77       384

+++++++++++++++++   HiBench  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.79      0.93      0.85       327
        True       0.70      0.40      0.51       137

    accuracy                           0.77       464
   macro avg       0.74      0.66      0.68       464
weighted avg       0.76      0.77      0.75       464

+++++++++++++++++   embulk  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.89      0.94      0.91       683
        True       0.47      0.30      0.37       115

    accuracy                           0.85       798
   macro avg       0.68      0.62  

              precision    recall  f1-score   support

       False       0.85      0.85      0.85       301
        True       0.65      0.65      0.65       133

    accuracy                           0.79       434
   macro avg       0.75      0.75      0.75       434
weighted avg       0.79      0.79      0.79       434

+++++++++++++++++   RedReader  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.85      0.81      0.83       372
        True       0.52      0.59      0.55       128

    accuracy                           0.76       500
   macro avg       0.69      0.70      0.69       500
weighted avg       0.77      0.76      0.76       500

+++++++++++++++++   BiglyBT  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.91      0.94      0.93       808
        True       0.50      0.39      0.44       118

    accuracy                           0.87       926
   macro avg       0.71      0.6

              precision    recall  f1-score   support

       False       0.78      0.73      0.75       381
        True       0.66      0.72      0.69       282

    accuracy                           0.73       663
   macro avg       0.72      0.72      0.72       663
weighted avg       0.73      0.73      0.73       663

+++++++++++++++++   chunky  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.77      0.85      0.81       201
        True       0.89      0.82      0.85       286

    accuracy                           0.83       487
   macro avg       0.83      0.84      0.83       487
weighted avg       0.84      0.83      0.83       487

+++++++++++++++++   dropwizard  +++++++++++++++++
              precision    recall  f1-score   support

       False       0.91      0.92      0.91      1357
        True       0.61      0.58      0.59       306

    accuracy                           0.85      1663
   macro avg       0.76      0.7

In [14]:
for key in final_result.keys():
    print(key,np.nanmedian(list(final_result[key].values())))

precision 0.51
recall 0.5
pf 0.26
f1 0.5
g 0.53
auc 0.5982142857142858


In [ ]:
_precision = list(precision_list.values())
_recall = list(recall_list.values())
_pf = list(pf_list.values())
_f1 = list(f1_list.values())
_g = list(g_list.values())
_auc = list(auc_list.values())
print(np.median(_precision),np.median(_recall),np.median(_pf),np.median(_f1),np.median(_g),np.median(_auc))
fig = plt.figure(num=None, figsize = (20,4), facecolor='w', edgecolor='k')
ax = fig.add_subplot(161)
ax.boxplot(_precision)
ax.set_title('Precision',size = 15)
ax = fig.add_subplot(162)
ax.boxplot(_recall)
ax.set_title('Recall',size = 15)
ax = fig.add_subplot(163)
ax.boxplot(_pf)
ax.set_title('pf',size = 15)
ax = fig.add_subplot(164)
ax.boxplot(_f1)
ax.set_title('f1',size = 15)
ax = fig.add_subplot(165)
ax.boxplot(_g)
ax.set_title('g',size = 15)
ax = fig.add_subplot(166)
ax.boxplot(_auc)
ax.set_title('auc',size = 15)
# fig.savefig('without_process.png')